In [0]:
# Define the configuration for mounting
spark.conf.set("spark.databricks.acl.data.frame.enforcement", "true")

# Define your ADLS details
storage_account_name = "<your_storage_account_name>"
container_name = "<your_container_name>"
mount_point = "/mnt/adls_mount"

# Use a service principal for authentication
client_id = dbutils.secrets.get(scope="adls-secrets", key="client-id")
client_secret = dbutils.secrets.get(scope="adls-secrets", key="client-secret")
tenant_id = dbutils.secrets.get(scope="adls-secrets", key="tenant-id")

# Set the necessary configs
spark.conf.set(f"spark.hadoop.fs.azure.account.auth.type.{storage_account_name}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"spark.hadoop.fs.azure.account.oauth.provider.type.{storage_account_name}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"spark.hadoop.fs.azure.account.oauth2.client.id.{storage_account_name}.dfs.core.windows.net", client_id)
spark.conf.set(f"spark.hadoop.fs.azure.account.oauth2.client.secret.{storage_account_name}.dfs.core.windows.net", client_secret)
spark.conf.set(f"spark.hadoop.fs.azure.account.oauth2.client.endpoint.{storage_account_name}.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")

# Mount the ADLS Gen2 container to Databricks
dbutils.fs.mount(
  source = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/",
  mount_point = mount_point,
  extra_configs = {"spark.hadoop.fs.azure" : "org.apache.hadoop.fs.azure.NativeAzureFileSystem"}
)
